In [1]:
from glob import glob 
import numpy as np 
import xarray as xr
import rioxarray

# find all nc files
files = glob("../data/raw/**/*.nc",recursive=True)

for f in files:
    print(f)

../data/raw/UKCP2.2/tasmax/01/latest/01/latest/tasmax_rcp85_land-cpm_uk_2.2km_01_day_19821201-19831130.nc
../data/raw/UKCP2.2/tasmax/01/latest/01/latest/tasmax_rcp85_land-cpm_uk_2.2km_01_day_19811201-19821130.nc
../data/raw/UKCP2.2/tasmax/01/latest/01/latest/tasmax_rcp85_land-cpm_uk_2.2km_01_day_19801201-19811130.nc


In [2]:
f0 = files[0]

## Test time requirement

In [3]:
#%timeit ncfile = xr.open_dataset(f0)

## Conversion

In [29]:
# Don't need the extra dims - AFAIK
ncfile = xr.open_dataset(f0).drop_dims(["bnds"]).squeeze()
v = ncfile['tasmax']
del v.attrs['grid_mapping'] # need to drop this too in order to size properly

In [20]:
#Define lat/long 
v = v.rio.set_spatial_dims('grid_longitude', 'grid_latitude')

# #(Optional) If your CRS is not discovered, you should be able to add it like so:
v = v.rio.write_crs("epsg:27700", inplace=True)


In [28]:
#Saving the file
v.rio.to_raster(f0.replace(".nc","geotiff.tiff"))